# 07. Molecular Visualization Test

## 분자 구조 시각화 테스트

**목적**: py3Dmol, nglview를 활용한 분자 구조 시각화

---

### 시각화 라이브러리

| 라이브러리 | 용도 |
|-----------|------|
| py3Dmol | 정적 구조 시각화 |
| nglview | 궤적 재생, 인터랙티브 |

## 1. 환경 설정

In [ ]:
import sys
from pathlib import Path
import numpy as np

# 프로젝트 루트 설정
PROJECT_ROOT = Path.cwd().parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import src
from ase.io import read, write

print(f"Project Root: {PROJECT_ROOT}")
print("✓ Modules loaded")

## 2. py3Dmol 테스트

정적 분자 구조 시각화

In [ ]:
import py3Dmol

def view_structure_py3dmol(cif_path, style='stick', width=600, height=400):
    """
    py3Dmol로 CIF 구조 시각화
    
    Parameters:
    -----------
    cif_path : str or Path
        CIF 파일 경로
    style : str
        'stick', 'sphere', 'line', 'cartoon'
    """
    # CIF 파일 읽기
    with open(cif_path, 'r') as f:
        cif_content = f.read()
    
    # 뷰어 생성
    view = py3Dmol.view(width=width, height=height)
    view.addModel(cif_content, 'cif')
    
    # 스타일 설정
    if style == 'stick':
        view.setStyle({'stick': {'radius': 0.1}})
    elif style == 'sphere':
        view.setStyle({'sphere': {'radius': 0.4}})
    elif style == 'ball_stick':
        view.setStyle({'stick': {'radius': 0.1}, 'sphere': {'radius': 0.3}})
    
    # 원소별 색상
    view.setStyle({'elem': 'Ca'}, {'sphere': {'color': 'green', 'radius': 0.5}})
    view.setStyle({'elem': 'Si'}, {'sphere': {'color': 'yellow', 'radius': 0.4}})
    view.setStyle({'elem': 'O'}, {'sphere': {'color': 'red', 'radius': 0.3}})
    view.setStyle({'elem': 'H'}, {'sphere': {'color': 'white', 'radius': 0.2}})
    view.setStyle({'elem': 'Al'}, {'sphere': {'color': 'pink', 'radius': 0.4}})
    view.setStyle({'elem': 'Fe'}, {'sphere': {'color': 'orange', 'radius': 0.45}})
    view.setStyle({'elem': 'Mg'}, {'sphere': {'color': 'cyan', 'radius': 0.4}})
    
    # 단위 셀 표시
    view.addUnitCell()
    
    view.zoomTo()
    view.setBackgroundColor('white')
    
    return view

print("✓ py3Dmol function defined")

In [ ]:
# C3S 최적화 구조 시각화
c3s_path = src.STRUCTURES_DIR / 'C3S_optimized.cif'

if c3s_path.exists():
    print(f"Visualizing: {c3s_path.name}")
    view = view_structure_py3dmol(c3s_path)
    view.show()
else:
    print(f"File not found: {c3s_path}")

In [ ]:
# Top 5 후보 구조 시각화
top5_names = ['FlyAshC', 'EAFSlag', 'WasteGlass', 'CopperSlag', 'SteelSlag']

for name in top5_names[:3]:  # Top 3만 표시
    cif_path = src.STRUCTURES_DIR / f'{name}_optimized.cif'
    if cif_path.exists():
        print(f"\n{'='*50}")
        print(f"{name} Structure")
        print(f"{'='*50}")
        view = view_structure_py3dmol(cif_path, width=500, height=350)
        view.show()

## 3. nglview 테스트

MD 궤적 시각화 및 재생

In [ ]:
import nglview as nv

def view_trajectory_nglview(traj_path, step=10):
    """
    nglview로 궤적 시각화
    
    Parameters:
    -----------
    traj_path : str or Path
        .traj 파일 경로
    step : int
        프레임 간격 (메모리 절약)
    """
    # 궤적 로딩 (일부 프레임만)
    traj = read(traj_path, index=f'::{step}')
    print(f"Loaded {len(traj)} frames (every {step}th frame)")
    
    # nglview 위젯 생성
    view = nv.show_asetraj(traj)
    
    # 스타일 설정
    view.clear_representations()
    view.add_ball_and_stick()
    
    # 배경 및 카메라
    view.background = 'white'
    view.camera = 'orthographic'
    
    return view

print("✓ nglview function defined")

In [ ]:
# C3S 수화 궤적 시각화
c3s_traj = src.TRAJECTORIES_DIR / 'csh_formation_10.0ps.traj'

if c3s_traj.exists():
    print(f"Loading trajectory: {c3s_traj.name}")
    view = view_trajectory_nglview(c3s_traj, step=100)
    view
else:
    print(f"File not found: {c3s_traj}")

In [ ]:
# Top 1 후보 (FlyAshC) 수화 궤적
flyash_traj = src.TRAJECTORIES_DIR / 'FlyAshC_hydration.traj'

if flyash_traj.exists():
    print(f"Loading trajectory: {flyash_traj.name}")
    view = view_trajectory_nglview(flyash_traj, step=100)
    view
else:
    print(f"File not found: {flyash_traj}")

## 4. ASE 기본 시각화

빠른 확인용

In [ ]:
from ase.visualize.plot import plot_atoms
import matplotlib.pyplot as plt

def view_structure_ase(cif_path, rotation='10x,10y,0z'):
    """
    ASE + Matplotlib로 구조 시각화
    """
    atoms = read(cif_path)
    
    fig, ax = plt.subplots(figsize=(8, 8))
    plot_atoms(atoms, ax=ax, rotation=rotation, radii=0.3)
    ax.set_title(cif_path.stem)
    ax.axis('off')
    plt.tight_layout()
    
    return fig, ax

# C3S 구조
if c3s_path.exists():
    fig, ax = view_structure_ase(c3s_path)
    plt.show()

## 5. 비교 시각화 (Side-by-Side)

In [ ]:
def compare_structures_ase(cif_paths, titles=None):
    """
    여러 구조를 나란히 비교
    """
    n = len(cif_paths)
    fig, axes = plt.subplots(1, n, figsize=(5*n, 5))
    
    if n == 1:
        axes = [axes]
    
    for i, (path, ax) in enumerate(zip(cif_paths, axes)):
        if Path(path).exists():
            atoms = read(path)
            plot_atoms(atoms, ax=ax, rotation='10x,10y,0z', radii=0.3)
            title = titles[i] if titles else Path(path).stem
            ax.set_title(title, fontsize=12, fontweight='bold')
        ax.axis('off')
    
    plt.tight_layout()
    return fig

# C3S vs Top 3 비교
paths = [
    src.STRUCTURES_DIR / 'C3S_optimized.cif',
    src.STRUCTURES_DIR / 'FlyAshC_optimized.cif',
    src.STRUCTURES_DIR / 'EAFSlag_optimized.cif',
]
titles = ['C3S (Baseline)', 'FlyAshC (#1)', 'EAFSlag (#2)']

fig = compare_structures_ase(paths, titles)
fig.savefig(src.FIGURES_DIR / 'paper' / 'structure_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Saved: structure_comparison.png")

## 6. 수화 전/후 비교

In [ ]:
# FlyAshC 수화 전/후 비교
paths = [
    src.STRUCTURES_DIR / 'FlyAshC_optimized.cif',
    src.STRUCTURES_DIR / 'FlyAshC_hydration.cif',
]
titles = ['FlyAshC (Optimized)', 'FlyAshC (+ H₂O)']

fig = compare_structures_ase(paths, titles)
fig.savefig(src.FIGURES_DIR / 'paper' / 'FlyAshC_hydration_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Saved: FlyAshC_hydration_comparison.png")

## 7. 요약

### 시각화 도구 사용법

| 도구 | 사용 상황 | 장점 |
|------|----------|------|
| **py3Dmol** | 정적 구조 | 3D 인터랙티브, CIF 직접 로딩 |
| **nglview** | MD 궤적 | 애니메이션 재생, 재료 과학에 적합 |
| **ASE plot** | 빠른 확인 | 간단, 이미지 저장 쉬움 |

### VMD 사용 (고품질 렌더링)

```bash
# Windows에서 VMD 설치 후:
# 1. File > New Molecule > .cif 또는 .xyz 로딩
# 2. Graphics > Representations 에서 스타일 조정
# 3. File > Render > Tachyon 으로 고품질 렌더링
```

### 다음 단계
- MatterGen 구조 생성 후 시각화
- 논문용 고품질 Figure 생성

In [ ]:
print("\n" + "="*60)
print("VISUALIZATION TEST COMPLETE")
print("="*60)
print("""
Available visualization methods:

1. py3Dmol  - view_structure_py3dmol(cif_path)
2. nglview  - view_trajectory_nglview(traj_path)
3. ASE plot - view_structure_ase(cif_path)

For publication-quality figures:
- Use VMD with Tachyon renderer
- Or OVITO for materials science

✅ Ready for Phase 7: MatterGen
""")